In [ ]:
# first, load the data in from a csv, pandas will work here
import pandas
import matplotlib
import numpy

In [ ]:
# after initial observations, start performing an eda on the dataset
# if date fields exist, may want to change them to a datetime format



In [ ]:
# for descriptive analysis look at central tendency measurements - mean, median, maybe mod
#   for variability look into standard deviation/variance/skewness
        # skewness can be found through using a histogram

In [ ]:
# for trends, look for if the data follows an exponential pattern.. etc
#   if exponential, it may benefit to take the log of the response variable
# for outliers, can use violin plot/box plot
# for correlations, can use a correlation matrix


In [ ]:
# transformations to the response variable can be done here
# I am not super familiar with all transformations, however, a common one that help lessen extremes in data is log

In [ ]:
# when creating the train/test split, use the line below, which enforces 30% of the data is considered testing data
# while 70% of the data is considered training data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
# the function call above is from scikit learn

In [ ]:
# for regression models, linear/polynomial/logistic will likely not be what we will care for, but who knows?
# decision trees/random forest, ridge/llasso.. etc

# scikit will be extremely helpful here.
# https://scikit-learn.org/stable/modules/preprocessing.html can be looked into for transformations and mputing of missing values

In [ ]:
# for cross validation, refer to: https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
# for hyper parameter tuning, refer to: https://scikit-learn.org/stable/modules/grid_search.html

In [ ]:
# team with lowest RMSE wins
#  https://www.geeksforgeeks.org/rmse-root-mean-square-error-in-matlab/